In [12]:
import numpy as np

def matmul_mod(a, b, m):
    return np.remainder(np.dot(a, b), m)

def pow_mat_mod(A, e, m, err_sq=None):
    R = np.identity(2, dtype=object)
    B = A.copy()
    mul_id = sq_id = 0
    while e:
        if e & 1:
            R = matmul_mod(R, B, m)
            mul_id += 1
        if err_sq is not None and sq_id == err_sq:
            B[0,0] = (B[0,0] + 123456789) % m
        B = matmul_mod(B, B, m)
        sq_id += 1
        e >>= 1
    return R

def gerbicz_check(A, power, m, r):
    q = power // r - 1
    X  = pow_mat_mod(A, r,         m)
    Y2 = pow_mat_mod(X, q + 1,     m)
    Y1 = pow_mat_mod(A, power,     m)
    ok = np.all(np.remainder(Y1 - Y2, m) == 0)
    return ok, Y1

def lucas_lehmer_mod(p, err_sq=None):
    Mp = (1 << p) - 1
    A  = np.array([[4, -1],[1, 0]], dtype=object)
    init = np.array([[4],[2]], dtype=object)
    power = 1 << (p - 2)
    r = 1 << ((p - 2)//2)

    ok, Apow_ref = gerbicz_check(A, power, Mp, r)
    if not ok:
        raise ValueError("Gerbicz reference failed")

    Apow_run = pow_mat_mod(A, power, Mp, err_sq=err_sq)
    if not np.all(np.remainder(Apow_run - Apow_ref, Mp) == 0):
        raise ValueError("Gerbicz run failed")

    v = matmul_mod(Apow_run, init, Mp)
    s = int(v[1,0])
    print(f"s_{p-2} mod M_{p} = {s}")
    print(f"2^{p}-1 = {Mp} is {'prime' if s==0 else 'composite'}")

lucas_lehmer_mod(13)

try:
    lucas_lehmer_mod(13, err_sq=3)
except ValueError as e:
    print(e)


try:
    lucas_lehmer_mod(13, err_sq=13)
except ValueError as e:
    print(e)

s_11 mod M_13 = 0
2^13-1 = 8191 is prime
Gerbicz run failed
s_11 mod M_13 = 0
2^13-1 = 8191 is prime


In [13]:
def run_case(p, err_sq, should_raise):
    try:
        lucas_lehmer_mod(p, err_sq=err_sq)
    except ValueError:
        if not should_raise:
            print("FAIL", p, err_sq, "unexpected error")
        else:
            print("OK", p, err_sq, "error detected")
        return
    if should_raise:
        print("FAIL", p, err_sq, "error NOT detected")
    else:
        print("OK", p, err_sq, "no error")

cases = [
    (13, None,  False),
    (13, 0,     True),
    (13, 1,     True),
    (13, 2,     True),
    (13, 3,     True),
    (13, 4,     True),
    (13, 5,     True),
    (13, 6,     True),
    (13, 7,     True),
    (13, 8,     True),
    (13, 9,     True),
    (13, 10,    True),
    (11, None,  False),
    (11, 0,     True),
    (11, 1,     True),
    (11, 2,     True),
    (11, 3,     True),
    (11, 4,     True),
    (11, 5,     True),
    (11, 6,     True),
    (17, None,  False),
    (17, 0,     True),
    (17, 1,     True),
    (17, 2,     True),
    (17, 3,     True),
    (17, 4,     True),
    (17, 5,     True),
    (17, 6,     True),
    (17, 7,     True),
    (17, 8,     True),
    (17, 9,     True),
    (17, 10,    True),
    (17, 11,    True),
    (17, 12,    True),
]

for p, err_sq, should_raise in cases:
    run_case(p, err_sq, should_raise)


s_11 mod M_13 = 0
2^13-1 = 8191 is prime
OK 13 None no error
OK 13 0 error detected
OK 13 1 error detected
OK 13 2 error detected
OK 13 3 error detected
OK 13 4 error detected
OK 13 5 error detected
OK 13 6 error detected
OK 13 7 error detected
OK 13 8 error detected
OK 13 9 error detected
OK 13 10 error detected
s_9 mod M_11 = 1736
2^11-1 = 2047 is composite
OK 11 None no error
OK 11 0 error detected
OK 11 1 error detected
OK 11 2 error detected
OK 11 3 error detected
OK 11 4 error detected
OK 11 5 error detected
OK 11 6 error detected
s_15 mod M_17 = 0
2^17-1 = 131071 is prime
OK 17 None no error
OK 17 0 error detected
OK 17 1 error detected
OK 17 2 error detected
OK 17 3 error detected
OK 17 4 error detected
OK 17 5 error detected
OK 17 6 error detected
OK 17 7 error detected
OK 17 8 error detected
OK 17 9 error detected
OK 17 10 error detected
OK 17 11 error detected
OK 17 12 error detected
